LIMPIEZA Y COMBINACION DE LAS BASES DE DATOS (PASO 1)

In [1]:
import pandas as pd

# Cargar archivos CSV
clientes = pd.read_csv('base_clientes_final.csv')
transacciones = pd.read_csv('base_transacciones_final.csv')

# Limpiar nombres de columnas por si acaso
clientes.rename(columns=lambda x: x.strip(), inplace=True)
transacciones.rename(columns=lambda x: x.strip(), inplace=True)

# Combinar las bases usando 'id',,,
base_completa = transacciones.merge(clientes, on='id', how='left')

# Llenar nulos en giro_comercio
base_completa['giro_comercio'].fillna("SIN CLASIFICAR", inplace=True)

# Vista previa
print("Vista previa de la base combinada:")
#print(base_completa.head())

# Guardar base combinada
base_completa.to_csv('base_completa.csv', index=False)
#print("\n¡Archivo guardado como 'base_completa.csv'!")

Vista previa de la base combinada:


DETECCIÓN DE GASTOS RECURRENTES (PASO 2)


In [2]:
# Asegurarse que la fecha sea tipo datetime 
base_completa['fecha'] = pd.to_datetime(base_completa['fecha'])

# Crear columna año-mes
base_completa['año_mes'] = base_completa['fecha'].dt.to_period('M')

# Agrupar por cliente y comercio para contar meses distintos
frecuencia_mensual = (
    base_completa.groupby(['id', 'comercio'])['año_mes']
    .nunique()
    .reset_index(name='meses_distintos')
)

continuacion parte 2 para las etiquetas

In [3]:
# Calcular estadísticas de monto
# Calcular promedio, std y número de transacciones por cliente-comercio
agrupado_montos = base_completa.groupby(['id', 'comercio'])['monto'].agg(
    monto_prom='mean',
    monto_std='std',
    num_transacciones='count'
).reset_index()

# Eliminar columnas duplicadas antes de hacer el merge para evitar conflictos de sufijos
cols_to_drop = ['monto_prom', 'monto_std', 'num_transacciones']
frecuencia_mensual = frecuencia_mensual.drop(columns=[col for col in cols_to_drop if col in frecuencia_mensual.columns])

# Unir con frecuencia_mensual
frecuencia_mensual = frecuencia_mensual.merge(agrupado_montos, on=['id', 'comercio'], how='left')

# Calcular rupturas de meses consecutivos
def contar_saltes(meses):
    meses_ordenados = sorted(meses.unique())
    diferencias = [meses_ordenados[i+1] - meses_ordenados[i] for i in range(len(meses_ordenados)-1)]
    return sum([d.n != 1 for d in diferencias])  # d.n convierte a valor numérico

rupturas = (
    base_completa.groupby(['id', 'comercio'])['año_mes']
    .apply(contar_saltes)
    .reset_index(name='meses_no_consecutivos')
)

# Unir con frecuencia_mensual
frecuencia_mensual = frecuencia_mensual.merge(rupturas, on=['id', 'comercio'], how='left')

# Crear etiquetas

# General: recurrente si cumple ≥7 meses, estabilidad en monto y pocos saltos
frecuencia_mensual['es_recurrente'] = (
    (frecuencia_mensual['meses_distintos'] >= 7) &
    (frecuencia_mensual['monto_std'] < 100) &
    (frecuencia_mensual['meses_no_consecutivos'] <= 2)
).astype(int)

# Fuerte: exactamente 12 meses de gasto
frecuencia_mensual['es_recurrente_fuerte'] = (
    frecuencia_mensual['meses_distintos'] == 12
).astype(int)

# Vista previa y exportación
#print(frecuencia_mensual.sort_values(by='meses_distintos', ascending=False).head())

frecuencia_mensual.to_csv('frecuencia_mensual_etiquetada.csv', index=False)
#print("\n¡Archivo guardado como 'frecuencia_mensual_etiquetada.csv'!")


PREDECIR COMERCIO Y MONTO SIGUIENTE (PASO 3)

In [4]:
frecuencia_mensual_etiquetada = pd.read_csv('frecuencia_mensual_etiquetada.csv')

base_modelo = base_completa.merge(
    frecuencia_mensual_etiquetada[['id', 'comercio', 'es_recurrente', 'es_recurrente_fuerte']], 
    on=['id', 'comercio'], 
    how='left'
)

# Filtrar solo los clientes recurrentes fuertes
base_recurrente = base_modelo[(base_modelo['es_recurrente'] == 1) & (base_modelo['es_recurrente_fuerte'] == 0)].copy()
base_recurrente = base_recurrente.sort_values(['id', 'fecha']) # Ordenar por cliente y fecha

# Shift por cliente: comercio y monto siguientes
# Calcular targets: comercio y monto siguientes
base_recurrente['comercio_siguiente'] = (
    base_recurrente.groupby('id')['comercio'].shift(-1)
)
base_recurrente['monto_siguiente'] = (
    base_recurrente.groupby('id')['monto'].shift(-1)
)

# Eliminar la última compra de cada cliente (no hay siguiente)
base_recurrente = base_recurrente.dropna(subset=['comercio_siguiente', 'monto_siguiente'])


Modelo clasificación comercio siguiente

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


# One-hot para giro_comercio (o comercio actual)
X = base_recurrente[['giro_comercio', 'monto']].copy()
X = pd.get_dummies(X, columns=['giro_comercio'])

y = base_recurrente['comercio_siguiente']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

print(f"Accuracy en test: {clf.score(X_test, y_test):.2f}")

Accuracy en test: 0.31


In [6]:
import pandas as pd
import numpy as np

# Asegúrate de que fecha es datetime
base_recurrente['fecha'] = pd.to_datetime(base_recurrente['fecha'])

# Ordenar por cliente y fecha
base_recurrente = base_recurrente.sort_values(['id', 'fecha'])

# Lags de montos
base_recurrente['monto_lag1'] = base_recurrente.groupby('id')['monto'].shift(1)
base_recurrente['monto_lag2'] = base_recurrente.groupby('id')['monto'].shift(2)

# Estadísticas históricas (cálculo expandido)
base_recurrente['monto_mean'] = base_recurrente.groupby('id')['monto'].expanding().mean().shift(1).reset_index(level=0, drop=True)
base_recurrente['monto_median'] = base_recurrente.groupby('id')['monto'].expanding().median().shift(1).reset_index(level=0, drop=True)
base_recurrente['monto_std'] = base_recurrente.groupby('id')['monto'].expanding().std().shift(1).reset_index(level=0, drop=True)

# Días desde última compra
base_recurrente['fecha_lag'] = base_recurrente.groupby('id')['fecha'].shift(1)
base_recurrente['dias_desde_ultima'] = (base_recurrente['fecha'] - base_recurrente['fecha_lag']).dt.days
# Crear dummies sin eliminar la columna original
dummies = pd.get_dummies(base_recurrente['giro_comercio'], prefix='giro_comercio', drop_first=True)

# Concatenar las dummies al DataFrame
base_recurrente = pd.concat([base_recurrente, dummies], axis=1)
base_modelo = base_recurrente.dropna(subset=[
    'monto_lag1', 'monto_lag2', 'monto_mean', 'monto_median', 'monto_std', 'dias_desde_ultima'
])
# Seleccionar columnas numéricas + one-hot de giro_comercio
columnas_modelo = [
    'monto_lag1', 'monto_lag2', 
    'monto_mean', 'monto_median', 'monto_std',
    'dias_desde_ultima'
] + [col for col in base_modelo.columns if col.startswith('giro_comercio_')]

X = base_modelo[columnas_modelo]


Modelo regresión monto siguiente

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import r2_score
import pandas as pd

y_monto = base_modelo['monto_siguiente']

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X, y_monto, test_size=0.2, random_state=42)

reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X_train_m, y_train_m)

preds = reg.predict(X_test_m)
mse = mean_squared_error(y_test_m, preds)
rmse = np.sqrt(mse)


print(f"RMSE del monto siguiente: {rmse:.2f}")
# Supón que ya tienes tu modelo entrenado y predicciones hechas
r2 = r2_score(y_test_m, preds)
print(f"R² del modelo: {r2:.4f}")

RMSE del monto siguiente: 38.15
R² del modelo: 0.1449


PREDECIR TIEMPO HASTA PRÓXIMA COMPRA (PASO 4)

In [8]:
# Asegurarse de que la fecha esté en formato datetime
base_completa = base_recurrente
base_completa['fecha'] = pd.to_datetime(base_completa['fecha'])

# Ordenar por cliente, comercio y fecha (¡fundamental!)
base_completa.sort_values(['id', 'comercio', 'fecha'], inplace=True)

# Crear columnas con la siguiente fecha y monto por cada pareja (id, comercio)
base_completa['fecha_siguiente'] = base_completa.groupby(['id', 'comercio'])['fecha'].shift(-1)
base_completa['monto_siguiente'] = base_completa.groupby(['id', 'comercio'])['monto'].shift(-1)

# Crear variable target: días hasta la siguiente transacción
base_completa['dias_hasta_siguiente'] = (base_completa['fecha_siguiente'] - base_completa['fecha']).dt.days
base_train = base_completa.dropna(subset=['dias_hasta_siguiente'])
base_completa['dias_entre_compras'] = base_completa.groupby(['id', 'comercio'])['fecha'].diff().dt.days

# Estadísticas temporales
estadisticas_tiempo = base_completa.groupby(['id', 'comercio'])['dias_entre_compras'].agg(
    media_dias_entre_compras='mean',
    mediana_dias_entre_compras='median'
).reset_index()

# Agregar estas estadísticas a base_completa
base_completa = base_completa.merge(estadisticas_tiempo, on=['id', 'comercio'], how='left')
#base_completa = base_completa.merge(estadisticas_tiempo2, on=['id', 'comercio'], how='left')

# --- Preparar datos para regresión de días ---

# Filtrar filas que tengan target definido (sin nulos)
base_train = base_completa.dropna(subset=['dias_hasta_siguiente'])

X = base_train[['monto', 'giro_comercio', 'fecha', 'media_dias_entre_compras', 'mediana_dias_entre_compras']].copy()

# Extraer variables de fecha para incluirlas como features
X['mes'] = X['fecha'].dt.month
X['dia_semana'] = X['fecha'].dt.dayofweek

# Eliminar columna fecha porque no la usaremos directamente
X.drop(columns=['fecha'], inplace=True)

# Convertir variable categórica 'giro_comercio' a variables dummy (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

# Variable objetivo
y = base_train['dias_hasta_siguiente']


Entrenar modelo

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=49)

rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=49)
rf.fit(X_train, y_train)

preds = rf.predict(X_test)
mae = mean_absolute_error(y_test, preds)
medae = median_absolute_error(y_test, preds)
print(f"MAE mejorado: {mae:.2f} días")
print(f"Median Absolute Error: {medae:.2f}")
r2 = r2_score(y_test, preds)

print(f"R² del modelo: {r2:.4f}")


MAE mejorado: 5.63 días
Median Absolute Error: 2.43
R² del modelo: 0.3631
